In [1]:
import os
import phoenix
import phoenix as px
import pandas as pd
import openai
import re
import nest_asyncio
nest_asyncio.apply()

In [2]:
# 將 PHOENIX_COLLECTOR_ENDPOINT 指向本機實際 IP
#os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://10.234.14.175:6006" #學校
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://phoenix:6006" #家
client = px.Client()
dataset = client.get_dataset(
    name="only3-dataset"
    # ,version_id="RGF0YXNldFZlcnNpb246MTE="
)

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (10.11.0) and client (10.12.0) versions are mismatched and may have compatibility issues.
  warnings.warn(


In [3]:
from google import genai

gemini_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
gemini_model = "gemini-2.0-flash"


def ask_gemini(contents):
    response = gemini_client.models.generate_content(
        model=gemini_model, contents=contents
    )
    return response.text    

In [9]:
def extract_brace_content_with_braces(s):
    start = s.find('{')
    end = s.rfind('}')
    if start != -1 and end != -1 and start < end:
        return s[start:end + 1]
    return None

In [10]:
export_records = [] #把資料存到資料表中

async def explain(example, prompt_template: str, model: str) -> str:
#     now = datetime.now()
#     time_str = now.strftime("%Y-%m-%d %H:%M:%S")
# #     global task_result
# #     task_result = time_str
    
#     time.sleep(2)
    
#     return "[TASK] " + time_str

    formatted_prompt_template = prompt_template.format(input_message=example.input["question"],output_message=example.output["respond"],reference_message=example.metadata["reference"])
    
    formatted_prompt_template = formatted_prompt_template + """

# Example
```json
{
  "result": "事實",
  "explaination": "水在標準大氣壓下的沸點是攝氏100度。"
}
```

```json
{
  "result": "幻覺",
  "explaination": "回答內容描述水的沸點是華氏100度，但應該是攝氏100度。"
}
```

"""
    
    content = ask_gemini(formatted_prompt_template)
    
    return extract_brace_content_with_braces(content)

In [11]:
import textwrap
from functools import partial

# template = """
# 你是一個判斷事實真偽的專家，請根據{reference_message}的內容，判斷{output_message}對於{input_message}的回答中，是否有與事實不符的地方。

# 以條列解釋你如何判斷為"事實"或"幻覺"
# """

template_json = """
# Role
你是一個判斷事實真偽的專家。

# Task
請根據{reference_message}的內容，判斷{output_message}對於{input_message}的回答中，是否有與事實不符的地方。

# Output
以JSON的型式，輸出以下資訊:
- result: "事實"或"幻覺"
- explaination: 你的解釋，以條列型式呈現，結合成一個字串。
"""

task = partial(explain, prompt_template=template_json, model="o1")
# print(dataset.examples)
test_example = dataset[0]
print(textwrap.fill(await task(test_example), width=100))

{   "result": "事實",   "explaination": "艾菲爾鐵塔確實位於法國巴黎。" }


In [12]:
from datetime import datetime

import json

def judge(metadata, output, input, reference, expected) -> str:
    try:
        # 嘗試解析 JSON 格式
        data = json.loads(output)
    except json.JSONDecodeError:
        # 若是單引號或非標準格式，轉換成可解析格式
        input_str = input_str.replace("'", '"')
        try:
            data = json.loads(input_str)
        except json.JSONDecodeError:
            return None  # 無法解析
    return data.get("result")

    # metadata, output, input, reference, expected.
    # now = datetime.now()
    # time_str = now.strftime("%Y-%m-%d %H:%M:%S")
    # return time_str
#     print({
#         "metadata": metadata,
#         "output": output,
#         "input": input,
#         "reference": reference,
#         "expected": expected,
#     })
    
#     return "[JUDGE]" + task_result
    # return output

    # =========================
    
#     input_message = input["question"]
#     output_message = expected["respond"]
#     reference_message = metadata["reference"]
#     # print(input_message)
#     # print(output_message)
#     # print(reference_message)
    
#     MY_CUSTOM_TEMPLATE = f'''

# [BEGIN DATA]
# [Question]: {input_message}
# [Response]: {output_message}
# [Reference]: {reference_message}
# [END DATA]

# # 角色
# 你是一個判斷事實真偽的專家，請根據[Reference]的內容，判斷[Response]對於[Question]的回答中，是否有與事實不符的地方。

# # 輸出格式
# "事實"或"幻覺"  

# '''
    
#     # print("==============================")
#     # print(MY_CUSTOM_TEMPLATE)
    
#     res = client.chat.completions.create(
#         model="o1",
#         temperature=0,
#         messages=[
#             {"role": "user", "content": MY_CUSTOM_TEMPLATE},
#         ]
#     )
#     content = res.choices[0].message.content
#     content = content.strip().lower()
#     return content

EVALUATORS = [judge]

In [13]:
from phoenix.experiments import run_experiment
experiment= run_experiment(
    dataset,
    task,
    experiment_name="explanation",
    experiment_metadata={"vendor": "openai", "model": "o1"},
    evaluators=EVALUATORS,
)

🧪 Experiment started.
📺 View dataset experiments: http://phoenix:6006/datasets/RGF0YXNldDoy/experiments
🔗 View this experiment: http://phoenix:6006/datasets/RGF0YXNldDoy/compare?experimentId=RXhwZXJpbWVudDoy


running tasks |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (10.11.0) and client (10.12.0) versions are mismatched and may have compatibility issues.
  warnings.warn(



🔗 View this experiment: http://phoenix:6006/datasets/RGF0YXNldDoy/compare?experimentId=RXhwZXJpbWVudDoy

Experiment Summary (06/16/25 09:30 PM +0000)
--------------------------------------------
  evaluator  n  n_labels        top_2_labels
0     judge  3         3  {'事實': 2, '幻覺': 1}

Tasks Summary (06/16/25 09:30 PM +0000)
---------------------------------------
   n_examples  n_runs  n_errors
0           3       3         0
